# Task 1

The aim of the first task is to determine the saturation voltage of the semiconductor detector. The measurement is performed in vacuum at the smallest possible distance $r_{rel} = 0$ mm. 

You recorded several histograms (spectrums) for different detector voltages $U_{Bias}$. 
Now we want to plot the peak positions and the detector resolution as a function of $U_{Bias}$ for Plutonium.
By looking at these plots, we can hopefully determine the saturation voltage and decide, which detector voltage to use for the rest of the experiment.

## Step 1: Look at the histogram

At first, we want to plot the raw histogram that you measured at $U_{Bias} = 100$ V.

In [ ]:
# The file v12_mca_lib.py contains some helper functions. We import them all:
from v12_mca_lib import check_files, load_asc_file, load_mcd_file

# Make sure that the filename here is correct:
datafile, metadatafile = check_files("data/task_01/hist_100v_00mm_vac.asc")
# The variable `histogram` is a 1D-array and contains the full histogram:
histogram = load_asc_file(datafile)
# From the metadata we need the "livetime" i.e. the duration in seconds that you recorded the data.
# The variable `metadata`is a dictionary that contains key-value pairs
metadata = load_mcd_file(metadatafile)

# We can access and print the livetime like this:
livetime = metadata['livetime']
print(f"livetime = {livetime} s")


Now that we have the data loaded into our program, we can plot it using pyplot:

In [ ]:
# enable the interactive features of matplotlib in jupyter notebook:
%matplotlib ipympl 
from matplotlib import pyplot as plt

fig1, ax1 = plt.subplots(figsize=(10, 6))
ax1.set_title("Measured Histogram")
ax1.plot(histogram, "-", linewidth=0.6)
ax1.grid(axis="both", which="both", linestyle="-", alpha=0.4)
ax1.set_xlabel("Channel No")
ax1.set_ylabel("Count")
plt.show()

## Step 2: Analyse the histogram

Next, we want to automatically locate the plutonium-peak in the spectrum and perform a gaussian-fit on it to determine the exact position and width.

For this we will use a peak finder and a fitting algorithm from the scipy package.

In [ ]:
from scipy.signal import find_peaks

# Run the peak-finding algorithm:
peaks_x, _ = find_peaks(histogram, distance=80, prominence=20)

# Calculate the y-values of the found peaks
peaks_y = [int(histogram[x]) for x in peaks_x]

# Print the result to the screen:
npeaks = peaks_x.size
print(f"Peakfinder found {npeaks} peaks in the histogram:")
print("  x     y")
for x, y, i in zip(peaks_x, peaks_y, range(npeaks)):
    print(f"{i} {x:4d}  {y:4d}")

In [ ]:
# plot the peaks onto the existing plot:
ax1.plot(peaks_x, peaks_y, "x")
plt.show()

The next step is fitting the plutonium peak, which is the first one in the list of peaks.

In [ ]:
import numpy
from scipy.optimize import curve_fit
from v12_mca_lib import gauss_func

hist_x = numpy.arange(histogram.size)
initial_params = numpy.array([peaks_y[0], peaks_x[0], 10])
params, pcov = curve_fit(gauss_func, hist_x, histogram, initial_params)
perr = numpy.sqrt(numpy.diag(pcov))

fit_x = hist_x[int(params[1] - 5 * params[2]) : int(params[1] + 5 * params[2])]
fit_y = gauss_func(fit_x, *params)

area_fit = 0
area_fit_err = 0
for x in fit_x:
    area_fit += histogram[x]
    area_fit_err += numpy.sqrt(histogram[x])

fwhm = 2 * numpy.sqrt(2 * numpy.log(2)) * params[2]
resolution = fwhm * 100 / params[1]

print(f"-- Fit peak --")
print(f"  A     = {params[0]:7.2f} {chr(0xb1)} {perr[0]:6.2f} counts")
print(f"  mu    = {params[1]:7.2f} {chr(0xb1)} {perr[1]:6.2f} channels")
print(f"  sigma = {params[2]:7.2f} {chr(0xb1)} {perr[2]:6.2f} channels")
print(f"  Z     = {area_fit / livetime:7.2f} {chr(0xb1)} {area_fit_err / livetime:6.2f} counts/s")
print(f"  res   = {resolution:.3f} %")

Now we can plot the fit result on top of the histogram:

In [ ]:
ax1.plot(fit_x, fit_y, "-")
plt.show()